### 1. AILA

#### 1.0 解决了什么问题
现有QA缺乏对法律知识的关注，无法提供较为准确的回答。

#### 1.1 数据如何获取的？如何处理的？如何储存的？

* 问答数据来自法律竞赛，法律问答论坛，律所。

* 知识图谱数据来自律所人工标记。

* 训练数据使用JSON格式处理，知识图谱采用 三元组形式 （Object -> Relation -> Entity）


#### 1.2 算法的大致框架

1. 使用 Bi-LSTM 对 问题 和 回答 进行表示。
2. 使用 MemoryNet，对问题和回答获取相关的知识图谱（回答这边是固定的，可以进行预处理）
3. 知识图谱 + 问题（回答）的拼接，成为最终的表示
4. 经过 Attention，然后进行检索评分，最后选出 TopN 的回答


#### 1.3 访问量很大怎么办

1. 使用消息系统对请求进行排队处理
2. 对可以预处理的数据进行预处理
3. 对服务器可以进行负载均衡

#### 1.4 如何评估这个系统的好坏

1. 检索式 -> TopN 评估，人工评估
2. 生成式 -> BLEU，人工评估

### 2. 设计

#### 2.1 设计一个请求响应系统

使用消息队列，对请求进行排队处理，防止请求过多，响应不及

#### 2.2 有一个 API 反应很慢怎么办？超时？

1. 异步处理，将等待API响应的事件放进队列，响应后得到通知再去处理
2. 设置计时器，对超时的进行重传，计时器的时间要根据失败率进行调整

#### 2.3 如何设计一个缓存？

1. 冷热分离，启动时缓存预热，将热数据加载到缓存中
2. 对缓存数据有效期设置一个随机区间，防止缓存雪崩
3. 对返回为空的结果也缓存，或者使用一个过滤器，防止频繁发生缓存穿透


#### 2.4 有一个超出内存的JSON文件该如何处理

这是一个什么样的文件，是多个JSON还说一个JSON，如果是一个JSON：

1. 流式处理，逐行读取每个 token，如一个 "[", "{"， 记录每个 token 对应值的存储位置（从第几行第几个位置到什么地方）
2. 构造出一颗解析树来
3. 根据解析树读取响应的数据